In [1]:
import os


DATA_DIR = 'data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings') # data is a directory where standings will be located inside
SCORES_DIR = os.path.join(DATA_DIR, 'scores')

In [2]:
SCORES_DIR

'data\\scores'

In [3]:
os.path.join(SCORES_DIR, 'compressed_games.zip')


'data\\scores\\compressed_games.zip'

In [4]:
file_names = os.listdir(SCORES_DIR)

In [12]:
import datetime


now = datetime.datetime.now()
ACTUAL_MONTH = str(now.month).zfill(2) # zfill is a method that adds a 0 to the left of the string if it has only one digit
ACTUAL_YEAR = str(now.year)

In [8]:
from bs4 import BeautifulSoup
import os
from playwright.sync_api import sync_playwright, TimeoutError as playwrightTimeout
import time
import datetime
import os, sys, shutil, gzip
import zlib
import zipfile
from tqdm import tqdm
from IPython.display import clear_output
import re


In [13]:
import time


def get_html(url, selector, sleep=7, retries=3): # async allow the code after it to imediatelly execute. 
    html=None
    for i in range(1, retries+1):
        time.sleep(sleep * i) # each try is longer by a sleep multiplication factor

        try:
            with sync_playwright() as p: # istance of playwright object
                browser = p.firefox.launch() # await will actually wait for the async load of the website complete to lauch the browser
                #context = browser.new_context()
                page = browser.new_page() # page will be a new tab
                page.goto(url)
                print(page.title()) # tracking our webscraping progress of tries and sucess
                html = page.inner_html(selector) # we gonna get only part of the html page
        except playwrightTimeout:
            print(f'TimeoutError on the url {url}')
            continue # tries again
        else: # if try suceds
            break
    if html != None:
        return html
    else:
        print('Fail')

In [ ]:

DATA_DIR = 'data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings') # data is a directory where standings will be located inside
SCORES_DIR = os.path.join(DATA_DIR, 'scores')
ACTUAL_MONTH_SCORES_DIR = os.path.join(DATA_DIR, 'actual_month_scores') # data is a directory where standings will be located inside
#SEASONS = list(range(2016, 2023))
SEASONS = [2018, 2019, 2020, 2021, 2022, 2023]

In [77]:
ACTUAL_MONTH = str(now.month).zfill(2)
SEASONS = [2023]
ACTUAL_MONTH_SCORES_DIR = os.path.join(DATA_DIR, 'actual_month_scores')

In [78]:
def scrape_boxscores(standing_file, season_year): # getting the paths of the htmls
    """ 
     Info:
        Gets the box scores of a month and saves it. We want to save all data in a directory,
        and the data of the actual month in a different directory, to deal with incoming real time data.
          Because the domain uses a single year to a single season that actually takes two years,
          we have to deal with the transition between seasons.
          For this, we are only saving the data of the actual month in the actual_month_scores directory, when
          the month == current month, and the year == the last year of the seasons list.
         --------------------------------
          Input:
             standing_file: path to the standings file (type: str)
              Example: 'data/standings/NBA_2022_games-june.html'
                season_year: year of the season (type: int)
               ----------------------------------------
                Output:
                   box_scores: list of box scores (type: list)
                     """
     # here we grab a month and go trough all the month box scores
    with open(standing_file,'r') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a')
    hrefs = [l.get('href') for l in links]
    box_scores = [href for href in hrefs if href and 'boxscore' in href and '.html' in href]
    box_scores = [f'https://www.basketball-reference.com/{href}' for href in box_scores ]
    #now we have all the links of the box , of all box of the months, of all months together ordered chronologically
    print(f'Now scraping the box scores of the {standing_file.split("/")[-1]}')

    for url in tqdm(box_scores): # navigate for each month page to save the file name first
            #if the file is from the actual month, its only saved on the actual month directory.
            #if does not exist, we can scrap and save it. For this to work,
            #we have to assume that the actual month is the last one in the list of box_scores
            #and is not in scores directory by some mistake

            #using regex to check if the url contains the actual month and year concatenated
            pattern = re.compile(f'{season_year}{ACTUAL_MONTH}')

            if re.search(pattern, url) and season_year == SEASONS[-1]: # if the url contains the actual month and year concatenated
                save_path = os.path.join(ACTUAL_MONTH_SCORES_DIR, url.split('/')[-1])
                print("matchs the actual month and year")
                print(f'saving {url} in {save_path}')
                continue
            else:
                save_path = os.path.join(SCORES_DIR, url.split('/')[-1]) # saving in directory the name of the BOX SCORE
                print("do not matchs the actual month and year")
                print(  f'saving {url} in {save_path}')
                print(pattern)
            
            if os.path.exists(save_path):# if already saved, do the next ( go out to start the loop)
                
                continue

          

            #grabbing table of that box score
            html = get_html(url=url, selector='#content') #grabing oly wha we want from id selector
            if not html:
                print(f'still nothing on the first attempt to get the whole {url} content')
                continue # if the link is broken or  laging we go on to the next 
            else:
                print(f'attempt to get the {url} content succeds!')
            
            # -*- coding: utf-8 -*-
            # trying to solve unicode error:
            #UnicodeEncodeError: 'charmap' codec can't encode character '\u0101' in position 346838: character maps to <undefined>
            
            #We are going to save the all the hmtl files in on directory, except the content of the actual month
            #This is because we are going to update the actual month every day, so dont need to compress it in th ecompressed_games.zip file.
            #We are going to save the actual month in a different directory, and we are going to compress it in a different zip file.
            
            with open (save_path, 'w+', encoding="utf-8") as f:
                f.write(html) # content to be saved in the file  with specified name



In [65]:
pattern = re.compile(f'{ACTUAL_YEAR}{ACTUAL_MONTH}')
pattern

re.compile(r'2023april', re.UNICODE)

In [52]:
if re.search(pattern, '928392832023Aprilsdhfusdfhsudf'):
    print('yes')

yes


In [80]:
SEASONS[-1]

2023

In [79]:
scrape_boxscores('data/standings/NBA_2023_games-april.html', 2023)

Now scraping the box scores of the NBA_2023_games-april.html


100%|██████████| 2/2 [00:00<?, ?it/s]

matchs the actual month and year
saving https://www.basketball-reference.com//boxscores/202304010MIA.html in data\actual_month_scores\202304010MIA.html
matchs the actual month and year
saving https://www.basketball-reference.com//boxscores/202304010NOP.html in data\actual_month_scores\202304010NOP.html


In [53]:
now = datetime.datetime.now()
#currenty month name
ACTUAL_MONTH = now.strftime("%B").lower()
ACTUAL_MONTH 

'april'

In [16]:
help(scrape_boxscores)

Help on function scrape_boxscores in module __main__:

scrape_boxscores(standing_file)

